# Purchase Order Item Categorization Challenge

In [1]:
# ==============================================================================
# SETUP: IMPORTING LIBRARIES
# ==============================================================================
# We'll start by importing the essential libraries for data manipulation, 
# visualization and other tasks as well. 

import pandas as pd  
import numpy as np   
import matplotlib.pyplot as plt 
import seaborn as sns 
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Set some visual styles for our plots to make them more readable.
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

In [6]:
# ==============================================================================
# Data Understanding
# ==============================================================================
# The following steps will be dedicted to understand the data to make the 
# data processing step as effective as possible. 
# EDA

df = pd.read_excel(r"C:\Users\yaman\Desktop\Personal Projects\PO Item Categorization Challenge\purchase-order-items.xlsx") # Load data to be read
display(df.head()) # Display the first 5 raws, just to understand generally how the data looks like
print("\n") # Space for better visibility
display(df.info()) # Let us understand more about the numerics (null instances, total rows, etc...)
print("\n") # Space for better visibility
# Let us focus on null values
missing_values = df.isnull().sum() # Calculate the number of missing values in each column.
if missing_values.empty: # If we have NO null-values
    print("Great! No missing values found in the dataset.")
else:
    print("Missing values found in the following columns:")
    missing_percentage = (missing_values / len(df)) * 100  # Calculate the percentage of missing values.
    missing_info = pd.DataFrame({'Missing Count': missing_values, 'Percentage (%)': missing_percentage}) # Put them as a data frame for the sake of sorting values by percentages
    print(missing_info.sort_values(by='Percentage (%)', ascending=False))




,Item ID,Item Name,Quantity,Total Bcy,Sub Total Bcy,Purchase Order ID,Product ID,Currency Code,Account ID,Tax ID,Project ID
0,3263831000000089042,NaN,75.21,227510.25,227510.25,3263831000000089040,NaN,SAR,3263831000000034003,3.263831e+18,NaN
1,3263831000000089046,NaN,15.00,47250.00,47250.00,3263831000000089040,NaN,SAR,3263831000000034003,3.263831e+18,NaN
2,3263831000000089044,NaN,50.00,160000.00,160000.00,3263831000000089040,NaN,SAR,3263831000000034003,3.263831e+18,NaN
3,3263831000000092044,NaN,12.00,38280.00,38280.00,3263831000000092040,NaN,SAR,3263831000000034003,3.263831e+18,NaN
4,3263831000000092052,NaN,2.00,5900.00,5900.00,3263831000000092040,NaN,SAR,3263831000000034003,3.263831e+18,NaN




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Item ID            3150 non-null   int64  
 1   Item Name          2910 non-null   object 
 2   Quantity           3150 non-null   float64
 3   Total Bcy          3150 non-null   float64
 4   Sub Total Bcy      3150 non-null   float64
 5   Purchase Order ID  3150 non-null   int64  
 6   Product ID         2910 non-null   float64
 7   Currency Code      3150 non-null   object 
 8   Account ID         3150 non-null   int64  
 9   Tax ID             3085 non-null   float64
 10  Project ID         0 non-null      float64
dtypes: float64(6), int64(3), object(2)
memory usage: 270.8+ KB


None



Missing values found in the following columns:
                   Missing Count  Percentage (%)
Project ID                  3150      100.000000
Item Name                    240        7.619048
Product ID                   240        7.619048
Tax ID                        65        2.063492
Item ID                        0        0.000000
Quantity                       0        0.000000
Total Bcy                      0        0.000000
Sub Total Bcy                  0        0.000000
Purchase Order ID              0        0.000000
Currency Code                  0        0.000000
Account ID                     0        0.000000


- Verbally, we can see that there are 11 features (columns) in our data frame, with total of 3150 instances (rows). a decent number to a certain extent. 

- Context understanding: 

    * A PO with count as a single purchasing process that can include mutiple items. This suggests that maybe one thing that we can do here is to look for highest POs in terms of the spending (look for unique values and then calculating the sum) which helps see highest categories sold.
    * I'm assuming that Item IDs are unique and do not change for the same item from one PO to another. One step that we will try is to recover missing Item Names using those if they exist in a different PO with their Item IDs. WHY? This is for us to be able to have a better conclusion afterwards on which items 
    * Bcy? I did not really understand this. So, instead of making a random assumption, I asked ChatGPT to suggest based on its knowledge, which it suggested to be (the total purchase value converted into the organization’s base (reporting) currency).
    * Total Bcy and Sub Total Bcy look to have the same content. I want to check if they are the same to remove one. Currently, I'm assuming they are different.

- <span style="color: red;">Targeted workflow</span>
    * <span style="color: blue;">Data Context Understanding</span>
    * <span style="color: blue;">Data Cleaning and Processing</span>
    * <span style="color: blue;">Results and Discussions</span>
    * <span style="color: blue;">Implementation (Data Analysis)</span>
    * <span style="color: blue;">Results and Discussions</span>
    * <span style="color: blue;">Conclusion</span>

- Initial steps to be taken --> 
    * It is evident that project ID must be eleminated as it has 100% null values. 
    * Try to recover missing Product names
    * Take decision to handle missing data

In [14]:
# Data Cleaning --> Remove Project ID
df1 = df.copy() # Make a copy just in case we need to come back to the original version
df1.drop("Project ID", inplace=True, axis=1) # Remove Project ID as it adds to value (100% null value)

In [ ]:
# Data Cleaning --> Try to recover item names

# In the following line, groups based on "Product IDs" and then from transform, we are bringing the
# first non null value from Item name in that group. Why? If I get something there, I know that we have
# Some instances that we can recover. Otherwise (in case if None), we know that this step 
# is not possible


nullMask = df1['Item Name'].isna() # I want to focus on null values (anything that haa values are irrelevant)
dfNull = df1[nullMask] # Only null values are here (based on Item Names)

df_named = df1.dropna(subset=['Item Name']) # In this instance, we are keeping the non null values, WHY?
                                            # This is the df where we will seek the Product ID for the present Item Names

name_counts = df_named.groupby('Product ID')['Item Name'].nunique() # Count instances of Item Name based on Product IDs
unique_one_ids = name_counts[name_counts == 1].index # Get Only the unique Product ID value? WHY???
# If the code above does NOT show unique Product IDs, this contradicts my initial assumption

# The following is AI generated to facilitate the mapping and reporting process of this
# experiment

single_name_map = ( # Create the mapping (the null Item name will now have the a name for the same Prodcut ID)
    df_named
      .drop_duplicates(subset=['Product ID','Item Name'])
      .set_index('Product ID')['Item Name']
      .loc[unique_one_ids]
)

# 3) For the null rows, see which are recoverable from the map
df_null_report = dfNull.assign(Proposed_Name=dfNull['Product ID'].map(single_name_map))
recoverable = df_null_report['Proposed_Name'].notna()

# --- Reporting ---
total_null = len(df_null_report)
can_recover = int(recoverable.sum())

print(f"Null Item Name rows: {total_null}")
print(f"Recoverable (same Product ID has a single known name elsewhere): {can_recover}")

print("\nExamples (up to 10):")
print(
    df_null_report.loc[recoverable, ['Product ID', 'Item Name', 'Proposed_Name']]
                   .drop_duplicates('Product ID')
                   .head(10)
                   .to_string(index=False)
)

Null Item Name rows: 240
Recoverable (same Product ID has a single known name elsewhere): 0

Examples (up to 10):
Empty DataFrame
Columns: [Product ID, Item Name, Proposed_Name]
Index: []


We can conculde that it is a bit difficult to recover this information. Hence and instead of adding some random names and product IDs, I will go with removing ALL null instances.

In [ ]:
# Data Cleaning, remove all null instances

df1.dropna(inplace=True) # Drop all null instances
display(df1.info()) # Check new standing
print()
display(df1.iloc[50:60,:]) # Check new standing

<class 'pandas.core.frame.DataFrame'>
Index: 2845 entries, 233 to 3149
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Item ID            2845 non-null   int64  
 1   Item Name          2845 non-null   object 
 2   Quantity           2845 non-null   float64
 3   Total Bcy          2845 non-null   float64
 4   Sub Total Bcy      2845 non-null   float64
 5   Purchase Order ID  2845 non-null   int64  
 6   Product ID         2845 non-null   float64
 7   Currency Code      2845 non-null   object 
 8   Account ID         2845 non-null   int64  
 9   Tax ID             2845 non-null   float64
dtypes: float64(5), int64(3), object(2)
memory usage: 244.5+ KB


None

,Item ID,Item Name,Quantity,Total Bcy,Sub Total Bcy,Purchase Order ID,Product ID,Currency Code,Account ID,Tax ID
286,3263831000000782410,HEB 260X260X10X17.5MMX12MTR,6.0,9288.72,9288.72,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
287,3263831000000782418,HEB 260X260X10X17.5MMX12MTR,144.0,2514.24,2514.24,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
288,3263831000000782426,"HDG ANGEL 50*50*5MM, 12MTR LENGTH CUTTING AS R...",6.0,5296.20,5296.20,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
289,3263831000000782406,HEB 260X260X10X17.5MMX12MTR,6.0,15166.92,15166.92,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
290,3263831000000782414,Expansion Bolt M12x150MM,6.0,3404.70,3404.70,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
291,3263831000000782422,Corrugated Sheet 1x6 mtr x 0.50 mm,12.0,873.00,873.00,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
292,3263831000000782412,GI Wire Mesh 12x12x1MM (ROLL),6.0,12222.00,12222.00,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
293,3263831000000782420,HEB 260X260X10X17.5MMX12MTR,144.0,4469.76,4469.76,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
294,3263831000000782408,Corrugated Sheet 1x6 mtr x 0.50 mm,12.0,22290.60,22290.60,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18
295,3263831000000782416,Expansion Bolt M12x150MM,72.0,4539.60,4539.60,3263831000000782404,3.263831e+18,SAR,3263831000000034003,3.263831e+18


In [ ]:
# 